# 📘 Chapter 18: Scaling Up with TensorFlow

## 🎯 Tujuan Bab

Bab ini berfokus pada bagaimana membangun model Machine Learning dengan **API tingkat rendah TensorFlow**, serta bagaimana mengoptimalkan dan mendistribusikan model untuk kebutuhan **deployment production-level**. Pendekatan ini memberi kontrol penuh terhadap proses training, optimasi, dan penyimpanan model.

---

## 🧩 Topik Utama

1. **SavedModel Format**
   - Format serialisasi standar TensorFlow untuk menyimpan dan mengekspor model.
   - Kompatibel dengan banyak tools lain: TensorFlow Serving, TensorFlow Lite, dan TensorFlow.js.

2. **@tf.function**
   - Mendekorasi fungsi Python biasa agar dikompilasi menjadi *static computation graph*.
   - Meningkatkan efisiensi eksekusi dan dapat dijalankan di perangkat keras khusus.

3. **Custom Training Loop**
   - Menggunakan `tf.GradientTape` untuk mengontrol proses forward-pass dan backward-pass secara manual.
   - Ideal untuk pelatihan fleksibel, fine-grained debugging, dan integrasi dengan logika bisnis yang kompleks.

4. **tf.data API**
   - Digunakan untuk membuat input pipeline yang scalable, efisien, dan dapat menangani data besar.
   - Mendukung batching, shuffling, caching, dan prefetching.

---

## ⚡ Manfaat Pendekatan Ini

- Cocok untuk **skala besar dan produksi**.
- Dapat **dioptimalkan secara manual** untuk performa, efisiensi memori, dan distribusi.
- Membuka pintu ke deployment real-world seperti:
  - Mobile apps (via TFLite)
  - Web (via TensorFlow.js)
  - Backend server (via TensorFlow Serving)

---

## 💡 Catatan Penting

> TensorFlow menyediakan banyak fleksibilitas melalui API tingkat rendah — namun dengan kekuatan tersebut datang juga kebutuhan untuk pemahaman lebih dalam tentang cara kerja training loop, optimizers, dan tensor operations.



In [2]:
# ✅ Import Library
import tensorflow as tf
from tensorflow import keras
import numpy as np

# ===================================================
# 🧠 Step 1: Membangun dan Melatih Model Sederhana
# ===================================================
(X_train, y_train), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_test = X_train / 255.0, X_test / 255.0

model = keras.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

model.fit(X_train, y_train, epochs=5, validation_split=0.1)

# ===================================================
# 💾 Step 2: Menyimpan dan Memuat Model (.keras format)
# ===================================================
model.save("my_fashion_model.keras")  # format baru Keras 3

# Load ulang model
new_model = keras.models.load_model("my_fashion_model.keras")
test_loss, test_acc = new_model.evaluate(X_test, y_test)
print(f"\n🎯 Accuracy model setelah load: {test_acc:.4f}")

# ===================================================
# 🔁 Step 3: TF Function - Kompilasi Fungsi ke Graph
# ===================================================
@tf.function
def scaled_add(x, y):
    return x + y * 2.0

x = tf.constant(2.0)
y = tf.constant(3.0)
print("\n🧮 Hasil scaled_add(2,3):", scaled_add(x, y).numpy())

# ===================================================
# 🔄 Step 4: Custom Training Loop dengan GradientTape
# ===================================================
def create_model():
    return keras.Sequential([
        keras.layers.Flatten(input_shape=[28, 28]),
        keras.layers.Dense(100, activation="relu"),
        keras.layers.Dense(10)
    ])

model = create_model()
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = keras.optimizers.SGD(learning_rate=0.01)

# Siapkan dataset
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(10000).batch(32)

# Training manual
for epoch in range(3):
    print(f"\n📦 Epoch {epoch+1}")
    for step, (x_batch, y_batch) in enumerate(train_ds):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fn(y_batch, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss_value.numpy():.4f}")


Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.6837 - loss: 1.0025 - val_accuracy: 0.7997 - val_loss: 0.5442
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8219 - loss: 0.5116 - val_accuracy: 0.8312 - val_loss: 0.4758
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8396 - loss: 0.4573 - val_accuracy: 0.8493 - val_loss: 0.4246
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8512 - loss: 0.4230 - val_accuracy: 0.8547 - val_loss: 0.4083
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8615 - loss: 0.3984 - val_accuracy: 0.8553 - val_loss: 0.4231
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8496 - loss: 0.4402

🎯 Accuracy model setelah load: 0.8424

🧮 Hasil scaled_add(2,3): 8.0

📦 Epoch 1
Step 0, Loss: 2.4420
Step 100, Loss: 1.2856
Step 200, Loss: 1.0221
Step 300, Loss: 0.7106
Step 400, Loss: 0.8575
Step 500, Loss: 0.8722
Step 600, Loss: 0.5978
Step 700, Loss: 0.5358
Step 800, Loss: 0.